In [1]:
##  QCEW data assembly

##  Eryk Wdowiak
##  29 July 2020

In [2]:
##  https://www.bls.gov/cew/downloadable-data-files.htm

In [3]:
import warnings
warnings.filterwarnings("ignore")
import glob
import pandas as pd

In [4]:
##  create empty dataframe for county statistics (calculated below)
cnty_cols = ['fips_code',  'state_code', 'mn_pay', 'sd_pay', 'cv_pay', 'nn_pay', 
             'pct_sp1011', 'pct_sp1012', 'pct_sp1013', 'pct_sp1021', 'pct_sp1022', 
             'pct_sp1023', 'pct_sp1024', 'pct_sp1025', 'pct_sp1026', 'pct_sp1027', 
             'pct_sp1028', 'pct_sp1029']
cnty_df = pd.DataFrame(columns = cnty_cols)

In [5]:
##  create lists of columns from the individual datafiles

##  initial first columns 
cols_first = ['own_code', 'industry_code', 'agglvl_code', 'size_code']

##  columns of interest
cols_int = ['annual_avg_estabs_count', 'annual_avg_emplvl',
            'avg_annual_pay','total_annual_wages']

##  columns to display
cols_disp = cols_first + cols_int

##  NOTE:  avg_annual_pay = total_annual_wages / annual_avg_emplvl

##  columns we don't need
##  ---------------------
##  'area_fips' -- FIPS area ID
##  'year' & 'qtr' -- all data is 2019 annual
##  'disclosure_code'
##  'taxable_annual_wages' -- may be zero (e.g. employer is US federal gov't)
##  'annual_contributions' -- may be zero
##  'annual_avg_wkly_wage' -- it's equal to:  avg_annual_pay / 52

In [6]:
##  get list of datafiles
extension = 'csv.gz'
qcew_files = glob.glob('qcew2019/*.{}'.format(extension))

##  read in the data
for qcew_file in sorted(qcew_files):
    
    ##  read in the datafile
    dta = pd.read_csv(qcew_file,compression='gzip')
    
    ##  only need first 20 (rest are location quotients and over year change)
    dta = dta[dta.columns[0:20]]
    
    ##  convert some columns to strings
    dta['area_fips'] = dta['area_fips'].apply(str)
    dta['own_code'] = dta['own_code'].apply(str)
    dta['industry_code'] = dta['industry_code'].apply(str)
    dta['agglvl_code'] = dta['agglvl_code'].apply(str)
    dta['size_code'] = dta['size_code'].apply(str)
    dta['year'] = dta['year'].apply(str)
    
    ##  get FIPS code and state code
    area_fips = dta['area_fips'][0]
    try:
        fips_code = f'{int(area_fips):05}'
    except:
        fips_code = area_fips
    state_code = fips_code[0:2]
    
    ##  create table for computation of county statistics
    indus6 = dta[dta['agglvl_code']=='78'].groupby('industry_code')[cols_int].sum()
    indus6 = indus6[indus6['annual_avg_emplvl'] > 0]
    
    ##  compute mean, std dev and coeff of variation
    nn_pay = indus6['annual_avg_emplvl'].sum()
    mn_pay = (indus6['avg_annual_pay'] * indus6['annual_avg_emplvl'] / nn_pay).sum()
    sd_pay = ((((indus6['avg_annual_pay'] - mn_pay)**2) * (indus6['annual_avg_emplvl'] / nn_pay)).sum())**0.5
    cv_pay = sd_pay / mn_pay
    
    ##  put them all in a series
    pay_stats = pd.Series(data = [mn_pay, sd_pay, cv_pay, nn_pay],
                          index = ['mn_pay', 'sd_pay', 'cv_pay', 'nn_pay'])
    
    ##  create table for computation of employment by supersector
    indus2 = dta[dta['agglvl_code']=='73'].groupby('industry_code')[cols_int].sum()
    
    ##  calculate percentages and put them in a series
    nn_mix = indus2['annual_avg_emplvl'].sum()
    pt_mix = indus2['annual_avg_emplvl'] / nn_mix
    pt_mix.index = ['pct_sp{0}'.format(idx) for idx in pt_mix.index]
    
    ##  get the identifiers
    id_codes = pd.Series([fips_code,state_code])
    id_codes.index = ['fips_code','state_code']
    
    ##  combine the three series
    cnty_row = pd.concat([id_codes,pay_stats,pt_mix])

    ##  give the series a name
    cnty_row.name = fips_code

    ##  if it is a county, then append the series to the county dataframe
    if fips_code.endswith('000'):
        blah = 'no good -- state'
    else:
        try:
            int(fips_code)
            ##  code is integer (therefore county), so append
            cnty_df = cnty_df.append(cnty_row)
        except:
            blah = 'no good -- US or MSA'


##  replace NA values with Zero in the supersector percentages
cnty_df[cnty_cols[6:18]] = cnty_df[cnty_cols[6:18]].fillna(0)

In [7]:
##  export the dataframe to CSV
cnty_df.to_csv('qcew2019.csv')